## Dynamic Task Offloading for Reactive LB

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
from threading import Thread
from queue import Queue
from time import sleep, perf_counter

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

### 1. Timer & Global Definition

In [2]:
# -----------------------------------------------------
# Constant definition
# -----------------------------------------------------
MIN_REL_LOAD_IMBALANCE = 0.05
PERCENT_DIFF_TASKS_TO_OFFLOAD = 0.05
THROUGHPUT = 0.25 # denotes 1 task/time unit
LATENCY = 2
DELAY = 2
TIMESTEP_RATIO = 10 # e.g., 1 wallclock exetime = 10 timesteps
NTASKS_OFFLOAD_AT_ONCE = 1

# -----------------------------------------------------
# Global status variables
# -----------------------------------------------------
global _time_step
global _stop_condition

### 2. Task Definition

In [3]:
class Task:
    """
    Class represent Task with its code entry, data, and childs (if yes).
    - task_id: unique id of each task
    - dur: duration or wallclock execution time of a task
    - data: data size or arguments of a task
    - childs: if yes
    - start_time: to be executed
    - end_time: to be termintated
    - is_offloaded_task: a task which is offloaded or migrated from process i to process j
    - off_time: offloaded time
    - arr_time: arrived time
    """
    
    def __init__(self, task_id, dur, data):
        self.task_id = task_id
        self.dur = dur
        self.data = data
        # other info that can be configured while queueing
        self.start_time = 0
        self.end_time = 0
        self.is_offloaded = False
        self.off_time = 0
        self.arr_time = 0
        
    def set_start_end_time(self, s_time, e_time):
        self.start_time = s_time
        self.end_time = e_time
    
    def set_offload_arrive_time(self, o_time, a_time):
        self.is_offloaded_task = True
        self.off_time = o_time
        self.arr_time = a_time
    
    def get_dur(self):
        return self.dur
        
    def get_end_time(self):
        return self.end_time
    
    def task_info(self):
        print('Task {}: dur({}), data({}), start_end_time({}-{}), is_offloaded({})'.format(self.task_id,
                self.dur, self.data, self.start_time, self.end_time, self.is_offloaded))
    

### 3. Task Execution and Load Update

In [4]:
# -----------------------------------------------------
# Util-functions
# -----------------------------------------------------
def load_task(queue, rank, timestep, mode, execu_task_arr,
              local_taskexe_arr, remot_taskexe_arr):
    cur_task = queue[rank].get()
    dur = cur_task.get_dur()
    s_time = timestep
    e_time = s_time + dur
    cur_task.set_start_end_time(s_time, e_time)
    execu_task_arr[rank] = cur_task
    # print('\t P{}: Task {} starts running!'.format(rank, cur_task.task_id))
    
    # record the task execution for tracking
    exe_interval = (s_time, dur)
    if mode == 'local':
        local_taskexe_arr[rank].append(exe_interval)
    elif mode == 'remote':
        remot_taskexe_arr[rank].append(exe_interval)

def update_load(r, status_task_done, timestep, execu_task_arr,
                local_load_arr, local_task_queue,
                remot_load_arr, remot_task_queue,
                local_taskexe_arr, remot_taskexe_arr):
    # check the task which is being executed
    if execu_task_arr[r] != None:
        # check the task is finished or not
        offloaded = execu_task_arr[r].is_offloaded
        end_time = execu_task_arr[r].get_end_time()
        if timestep == end_time:
            # print('\t P{}: Task {} is finishing...'.format(r, execu_task_arr[r].task_id))
            execu_task_arr[r] = None
        # else:
            # print('\t P{}: Task {} is running...'.format(r, execu_task_arr[r].task_id))
            
        # update the total load
        if offloaded == True:
            remot_load_arr[r] += 1
        else:
            local_load_arr[r] += 1
                
    # try to load task for executing when the array is empty
    if execu_task_arr[r] == None:
        # if remot_task_queue has some tasks | priority 1
        if remot_task_queue[r].qsize() > 0:
            load_task(remot_task_queue, r, timestep, 'remote', execu_task_arr, local_taskexe_arr, remot_taskexe_arr)
        elif local_task_queue[r].qsize() > 0:
            load_task(local_task_queue, r, timestep, 'local', execu_task_arr, local_taskexe_arr, remot_taskexe_arr)
        else:
            # assign the global stop condition
            status_task_done[r] = True

In [5]:
# -----------------------------------------------------
# Summarize load information at the end
# -----------------------------------------------------
def load_info_statistic(local_load_arr, remot_load_arr):
    print('------------------------------------------------')
    print('Local load: {}'.format(local_load_arr))
    print('Remot load: {}'.format(remot_load_arr))
    
    TOTAL_LOAD_ARR = np.add(local_load_arr, remot_load_arr)
    print('Total load: {}'.format(TOTAL_LOAD_ARR))
    print('Wallclock exetime: {}'.format(np.max(TOTAL_LOAD_ARR)))
    print('------------------------------------------------')

### 4. Reactive Load Balancing

In [6]:
# -----------------------------------------------------
# Offload & Receive tasks
# -----------------------------------------------------
def offload_tasks(src, des, ntasks, toffload, tarrive, loctask_queue, offtask_buff):
    # dequeue tasks from src
    offloaded_task = loctask_queue[src].get()
    offloaded_task.is_offloaded = True
    # set task info 
    migratabl_task = offloaded_task
    migratabl_task.set_offload_arrive_time(toffload, tarrive)
    # enqueue tasks to the offload-communication queue
    offtask_buff[des].append(migratabl_task)
    # print("[Offload] OFFLO_TASK_BUFFE[R{}], migratabl_task={}".format(des, migratabl_task.task_id))

def receive_tasks(offtask_buff, remottask_queue, num_ranks, timestamp):
    for i in range(num_ranks):
        if len(offtask_buff[i]) > 0:
            for j in range(len(offtask_buff[i])):
                tmp_task = offtask_buff[i][j]
                tarrive = tmp_task.arr_time
                if tarrive == timestamp:
                    remote_task = offtask_buff[i].pop(j)
                    # add remote task to the queue
                    remottask_queue[i].put(remote_task)
                    # print("[Receiv] REMOT_TASK_QUEUE[R{}], remote_task={}".format(i, remote_task.task_id))
                
# -----------------------------------------------------
# Balancing stuff
# -----------------------------------------------------
def balancing(timestep, local_task_queue, offload_task_buffer, num_ranks):
    # check local queue status
    QUEUE_SIZE_STATUS_ARR = np.zeros(num_ranks)
    for r in range(num_ranks):
        QUEUE_SIZE_STATUS_ARR[r] = local_task_queue[r].qsize()

    # sort the queues by the current size status
    sortLCQ = np.argsort(QUEUE_SIZE_STATUS_ARR)
  
    # show the status
    str_rank_orders = '[ '
    for i in range(len(sortLCQ)):
        str_rank_orders += 'R' + str(sortLCQ[i]) + ' '
    str_rank_orders += ']'
  
    str_load_orders = '[ '
    for i in range(len(sortLCQ)):
        idx = sortLCQ[i]
        str_load_orders += str(local_task_queue[idx].qsize()) + ' '
    str_load_orders += ']'

    # calculate the imbalance ratio
    lmax = QUEUE_SIZE_STATUS_ARR[sortLCQ[-1]]
    lmin = QUEUE_SIZE_STATUS_ARR[sortLCQ[0]]
    lavg = np.average(QUEUE_SIZE_STATUS_ARR)
    rimb = 0.0
    rimb_min_max = 0.0
    if lavg != 0 and lmax != 0:
        rimb = lmax/lavg - 1
        rimb_min_max = (lmax - lmin) / lmax

    # check the imbalance ratio
    if rimb_min_max >= MIN_REL_LOAD_IMBALANCE:
        # print(str_rank_orders + ' = ' + str_load_orders + ' | Imb. = ' + str(rimb))
        # print('Ratio (max, min): {}'.format(rimb_min_max))

        # calculate tasks to offload
        OFFLOAD_PAIR_RANKS = []
        for i in range(num_ranks):
            src_ntasks = [-1,-1]
            OFFLOAD_PAIR_RANKS.append(src_ntasks)

        for i in range(int(num_ranks/2), num_ranks):
            # check the number of tasks in src. rank
            cur_qsize = local_task_queue[sortLCQ[i]].qsize()
            if cur_qsize > 2:
                OFFLOAD_PAIR_RANKS[i][0] = sortLCQ[i] # src. rank
                OFFLOAD_PAIR_RANKS[i][1] = sortLCQ[num_ranks - i - 1] # des. rank
                # print("[Debug] src_rank={}, des_rank={}...".format(OFFLOAD_PAIR_RANKS[i][0], OFFLOAD_PAIR_RANKS[i][1]))

        # offload tasks
        for i in range(num_ranks):
            if np.sum(OFFLOAD_PAIR_RANKS[i]) > 0:
                src_rank = OFFLOAD_PAIR_RANKS[i][0]
                des_rank = OFFLOAD_PAIR_RANKS[i][1]
                ntasks2offload = NTASKS_OFFLOAD_AT_ONCE
                t_offloa = timestep
                t_arrive = timestep + int(ntasks2offload/THROUGHPUT)
                # print("[Offload] R{} ---> R{}: {} task, send at t{}, recv at t{}".format(src_rank, des_rank, ntasks2offload, t_offloa, t_arrive))
                offload_tasks(src_rank, des_rank, ntasks2offload, t_offloa, t_arrive, local_task_queue, offload_task_buffer)

### 5. Load Init & Simulator Configuration

In [7]:
# -----------------------------------------------------
# Init offloading tracker array
# -----------------------------------------------------
def init_configuration(given_task_distribution, perf_slowdown_sets, timestep_ratio):
    
    # Total load recoder
    GIVEN_TASK_ARR = given_task_distribution
    PERFO_SLOWDOWN = perf_slowdown_sets
    TIMESTEP_RATIO = timestep_ratio
    num_ranks = len(GIVEN_TASK_ARR)
    
    LOCAL_LOAD_ARR = np.zeros((num_ranks,), dtype=int)
    REMOT_LOAD_ARR = np.zeros((num_ranks,), dtype=int)
    
    # Offloading and task queues
    EXECU_TASK_ARRAY = []
    OFFLO_TASK_BUFFE = []
    LOCAL_TASK_QUEUE = []
    REMOT_TASK_QUEUE = []
    # Array tracking task execution for visualization
    LOCAL_TASKEXE_ARR = []
    REMOT_TASKEXE_ARR = []

    for i in range(len(GIVEN_TASK_ARR)):
        OFFLO_TASK_BUFFE.append([])
        LOCAL_TASK_QUEUE.append(Queue())
        REMOT_TASK_QUEUE.append(Queue())

        LOCAL_TASKEXE_ARR.append([])
        REMOT_TASKEXE_ARR.append([])

    for r in range(len(GIVEN_TASK_ARR)):
        num_given_tasks = GIVEN_TASK_ARR[r]
        r_to_tid = r * 10000 # to get a uniqe id for each task
        for i in range(num_given_tasks):
            tid = r_to_tid + i
            dur = 1*TIMESTEP_RATIO*PERFO_SLOWDOWN[r]
            data = 1 # such as 1MB
            task = Task(tid, dur, data)
            # put task to the queue
            LOCAL_TASK_QUEUE[r].put(task)
        
    # Check the number of generated tasks on each process
    # for r in range(len(GIVEN_TASK_ARR)):
    #     print('Process {:2d}: {:3d} tasks, load/task ~{:5.1f}(time-unit)'.format(r,
    #                     LOCAL_TASK_QUEUE[r].qsize(), 1*TIMESTEP_RATIO*PERFO_SLOWDOWN[r]))
        
    ret = [GIVEN_TASK_ARR,
          LOCAL_LOAD_ARR, REMOT_LOAD_ARR,
          EXECU_TASK_ARRAY, OFFLO_TASK_BUFFE,
          LOCAL_TASK_QUEUE, REMOT_TASK_QUEUE,
          LOCAL_TASKEXE_ARR, REMOT_TASKEXE_ARR]
    return ret
        
# Test the function
# tmp = init_configuration([5,5,5,5,5,5,5,5], [0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0], 10)

-----------------------------------------------------
#### Visualize Task Execution without Load Balancing
* $L_{i}$: denotes the total load of Process $i$
* $w_{i}$: denotes the wallclock exetime of each task
* With a given distribution of tasks, $T_{i}$ indicates the set of tasks belonging to Process $i$
* The model would be: $$ L(i) = \sum_{j \in T_{i}} w_{j} $$
-----------------------------------------------------


In [14]:
def segment_timestamp(total_load_arr):
    gnt_arr = []
    for r in range(len(total_load_arr)):
        tmp_arr = []
        for i in range(len(total_load_arr[r])):
            load_vals = total_load_arr[r]
            start = i * load_vals[i]
            dur = load_vals[i]
            exe_interval = (start, dur)
            tmp_arr.append(exe_interval)
        gnt_arr.append(tmp_arr)
    return gnt_arr

def visualize_load_no_lb(ntaskp0, ntaskp1, ntaskp2, ntaskp3, ntaskp4, ntaskp5, ntaskp6, ntaskp7,
                         pslowp0, pslowp1, pslowp2, pslowp3, pslowp4, pslowp5, pslowp6, pslowp7):
    task_arr = [ntaskp0, ntaskp1, ntaskp2, ntaskp3, ntaskp4, ntaskp5, ntaskp6, ntaskp7]
    perfslowdown_arr = [pslowp0, pslowp1, pslowp2, pslowp3, pslowp4, pslowp5, pslowp6, pslowp7]
    max_ntasks = np.max(task_arr)
    num_processes = len(task_arr)
    total_load_arr = []
    for i in range(num_processes):
        n_tasks = task_arr[i]
        load_per_task = 1 * TIMESTEP_RATIO * perfslowdown_arr[i]
        load_per_rank = [load_per_task] * n_tasks
        total_load_arr.append(load_per_rank)
        # print('P{}: {}'.format(i, load_per_rank))
    
    # plot the gannt chart
    fig, gnt = plt.subplots()
    
    # set labels for x- and y-axis
    gnt.set_xlabel('Time Progress')
    gnt.set_ylabel('Processes')
    
    # set x- or y-limits
    gnt.set_xlim(0, max_ntasks*TIMESTEP_RATIO*1.0+TIMESTEP_RATIO*2)
    
    # set ticks on y-axis for showing the process-names
    ytick_values = [15]
    ytick_labels = ['P0']
    for i in range(1, num_processes):
        ytick_values.append(ytick_values[i-1] + 10)
        ytick_labels.append('P' + str(i))
    gnt.set_yticks(ytick_values)
    gnt.set_yticklabels(ytick_labels)
    
    # configure the graph attributes
    # gnt.grid(True)
    
    # segment task timestamp
    gnt_load_arr = segment_timestamp(total_load_arr)
    print(gnt_load_arr)
    
    # declare bars in schedule
    for r in range(num_processes):
        gnt.broken_barh(gnt_load_arr[r], (10*r+10, 8), facecolors=('tab:green'), edgecolor='black')

    # display the chart
    plt.show()

# check the function
# visualize_load_no_lb(5, 5, 5, 5, 5, 5, 5, 5,
#                      0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0)

In [15]:
exetask_withoutlb_demo = interactive(visualize_load_no_lb,
                        ntaskp0=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P0'),
                        ntaskp1=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P1'),
                        ntaskp2=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P2'),
                        ntaskp3=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P3'),
                        ntaskp4=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P4'),
                        ntaskp5=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P5'),
                        ntaskp6=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P6'),
                        ntaskp7=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P7'),
                        pslowp0=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P0'),
                        pslowp1=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P1'),
                        pslowp2=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P2'),
                        pslowp3=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P3'),
                        pslowp4=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P4'),
                        pslowp5=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P5'),
                        pslowp6=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P6'),
                        pslowp7=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P7'))
display(exetask_withoutlb_demo)

interactive(children=(IntSlider(value=5, description='# tasks P0', max=50, min=5, step=5), IntSlider(value=5, …

### 6. Reactive Load Balancing Simulation Engine

In [10]:
# -----------------------------------------------------
# Main simulation engine
# -----------------------------------------------------
global _time_step
global _stop_condition
global _stat_all_tasks_done

def simulation_engine(given_task_distribution, perf_slowdown_sets, timestep_ratio):
    
    # refresh the arrays
    ret_arrays = init_configuration(given_task_distribution, perf_slowdown_sets, timestep_ratio)

    GIVEN_TASK_ARR = ret_arrays[0]
    LOCAL_LOAD_ARR = ret_arrays[1]
    REMOT_LOAD_ARR = ret_arrays[2]
    EXECU_TASK_ARRAY = ret_arrays[3]
    OFFLO_TASK_BUFFE = ret_arrays[4]
    LOCAL_TASK_QUEUE = ret_arrays[5]
    REMOT_TASK_QUEUE = ret_arrays[6]
    LOCAL_TASKEXE_ARR = ret_arrays[7]
    REMOT_TASKEXE_ARR = ret_arrays[8]
    
    _num_ranks = len(GIVEN_TASK_ARR)
    _time_step = 0
    _stop_condition = False
    
    # load the 1st tast into the execution buffer
    for r in range(_num_ranks):
        first_task = LOCAL_TASK_QUEUE[r].get()
        dur = first_task.dur
        s_time = 0
        e_time = s_time + dur
        first_task.set_start_end_time(s_time, e_time)
        EXECU_TASK_ARRAY.append(first_task)
        # record the 1st task for tracking
        exe_1st_task_interval = (s_time, dur)
        LOCAL_TASKEXE_ARR[r].append(exe_1st_task_interval)

    # set the stop condition flags on each process
    _stat_all_tasks_done = []
    for r in range(_num_ranks):
        _stat_all_tasks_done.append(0)

    # main loop
    while _stop_condition != True:
        # increase time clock
        _time_step = _time_step + 1
        # print('------------------------------------------------')
        # print('_time_step {}:'.format(_time_step))

        # check and recieve offload tasks
        receive_tasks(OFFLO_TASK_BUFFE, REMOT_TASK_QUEUE, _num_ranks, _time_step)

        # dynamic balancing
        balancing(_time_step, LOCAL_TASK_QUEUE, OFFLO_TASK_BUFFE, _num_ranks)

        # update load/proces as parallel processing
        for r in range(_num_ranks):
            # for process r at timestep t
            update_load(r, _stat_all_tasks_done, _time_step, EXECU_TASK_ARRAY,
                        LOCAL_LOAD_ARR, LOCAL_TASK_QUEUE,
                        REMOT_LOAD_ARR, REMOT_TASK_QUEUE,
                        LOCAL_TASKEXE_ARR, REMOT_TASKEXE_ARR)

        # check the stop condition for the main loop
        if sum(_stat_all_tasks_done) == _num_ranks:
            _stop_condition = True
            load_info_statistic(LOCAL_LOAD_ARR, REMOT_LOAD_ARR)
    
    return LOCAL_TASKEXE_ARR, REMOT_TASKEXE_ARR


In [11]:
# Test the simulation engine
simulation_engine([5,5,5,5,5,5,5,5], [0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0], 30)

------------------------------------------------
Local load: [ 60  60  60  60 120 120 120 120]
Remot load: [30 30 30 30 15 15 15 15]
Total load: [ 90  90  90  90 135 135 135 135]
Wallclock exetime: 135
------------------------------------------------


([[(0, 15.0), (15, 15.0), (60, 15.0), (75, 15.0)],
  [(0, 15.0), (15, 15.0), (60, 15.0), (75, 15.0)],
  [(0, 15.0), (15, 15.0), (60, 15.0), (75, 15.0)],
  [(0, 15.0), (15, 15.0), (60, 15.0), (75, 15.0)],
  [(0, 30.0), (30, 30.0), (75, 30.0), (105, 30.0)],
  [(0, 30.0), (30, 30.0), (75, 30.0), (105, 30.0)],
  [(0, 30.0), (30, 30.0), (75, 30.0), (105, 30.0)],
  [(0, 30.0), (30, 30.0), (75, 30.0), (105, 30.0)]],
 [[(30, 30.0)],
  [(30, 30.0)],
  [(30, 30.0)],
  [(30, 30.0)],
  [(60, 15.0)],
  [(60, 15.0)],
  [(60, 15.0)],
  [(60, 15.0)]])

In [12]:

def visualize_load_react_lb(ntaskp0, ntaskp1, ntaskp2, ntaskp3, ntaskp4, ntaskp5, ntaskp6, ntaskp7,
                         pslowp0, pslowp1, pslowp2, pslowp3, pslowp4, pslowp5, pslowp6, pslowp7,
                         time_ratio, offload_throughput):
    # prepare inputs
    given_task_dist = [ntaskp0, ntaskp1, ntaskp2, ntaskp3, ntaskp4, ntaskp5, ntaskp6, ntaskp7]
    perf_slowdown_dist = [pslowp0, pslowp1, pslowp2, pslowp3, pslowp4, pslowp5, pslowp6, pslowp7]
    THROUGHPUT = offload_throughput
    num_processes = len(given_task_dist)
    
    # run simulation
    local_task_arr, remot_task_arr = simulation_engine(given_task_dist, perf_slowdown_dist, time_ratio)
    total_load_arr = np.add(local_task_arr, remot_task_arr)
    max_load = np.max(total_load_arr)
    max_x = 100
    if max_load >= 100:
        max_x = max_load + 20
    
    # plot the gannt chart
    fig, gnt = plt.subplots()
    
    # set labels for x- and y-axis
    gnt.set_xlabel('Time Progress')
    gnt.set_ylabel('Processes')
    
    # set x- or y-limits
    gnt.set_xlim(0, max_x)
    
    # set ticks on y-axis for showing the process-names
    ytick_values = [15]
    ytick_labels = ['P0']
    for i in range(1, num_processes):
        ytick_values.append(ytick_values[i-1] + 10)
        ytick_labels.append('P' + str(i))
    gnt.set_yticks(ytick_values)
    gnt.set_yticklabels(ytick_labels)
    
    # configure the graph attributes
    # gnt.grid(True)
    
    # declare bars in schedule
    for r in range(num_processes):
        gnt.broken_barh(local_task_arr[r], (10*r+10, 8), facecolors=('tab:green'), edgecolor='black')
    for r in range(num_processes):
        gnt.broken_barh(remot_task_arr[r], (10*r+10, 8), facecolors=('tab:orange'), edgecolor='black')

    # display the chart
    plt.show()
    

In [13]:
exetask_withlb_demo = interactive(visualize_load_react_lb,
                        ntaskp0=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P0'),
                        ntaskp1=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P1'),
                        ntaskp2=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P2'),
                        ntaskp3=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P3'),
                        ntaskp4=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P4'),
                        ntaskp5=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P5'),
                        ntaskp6=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P6'),
                        ntaskp7=widgets.IntSlider(min=5, max=50, value=5, step=5, description='# tasks P7'),
                        pslowp0=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P0'),
                        pslowp1=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P1'),
                        pslowp2=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P2'),
                        pslowp3=widgets.FloatSlider(min=0.1, max=1.0, value=0.5, step=0.1, description='Perf-Slowdown P3'),
                        pslowp4=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P4'),
                        pslowp5=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P5'),
                        pslowp6=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P6'),
                        pslowp7=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, description='Perf-Slowdown P7'),
                        time_ratio=widgets.IntSlider(min=5, max=100, value=10, step=5, description='Timestep Ratio'),
                        offload_throughput=widgets.FloatSlider(min=0.05, max=100.0, value=0.5, step=0.05, description='Offload Throughput'))
display(exetask_withlb_demo)

interactive(children=(IntSlider(value=5, description='# tasks P0', max=50, min=5, step=5), IntSlider(value=5, …